In [32]:
import os
import numpy as np
from plotly.offline import init_notebook_mode, iplot
import plotly.graph_objects as go

init_notebook_mode(connected=True)


' Assumptions made: (9/4/20)\n        - This is specifically for Atlas IR\'\n        - We are not using specFile for now\n        - This is what userinput looks like:\n         "time":2*3600, # exposure time in seconds \n            "slit_width":2.8,  # width of slit in arcseconds \n                - made by DMDs (refers to the actual width of the slit, length "doesnt matter")\n            "Nreads":16, # number of reads in fowler - "fowler sampling in IR detectors"\n                - this is in a sense equivalent to CCD and binning; basically 16 refers to the number of sample being taken, \n                    which is separate from the exposure time (image like, 16 samples/second, for 5 seconds)\n                - multiple reads of this are taken (meaning multiple exposures in the same place) so that artifacts like cosmic rays\n                    and other noise can be filterd out through averaging\n            "theta":1.4, # angular size of the object along the slit \n              

In [33]:
#reading the files

homedir = os.path.join(os.getcwd(),"etc_data")
throughput = np.loadtxt(os.path.join(homedir,"spec_throughput","throughputSpec_atlas_IR.txt"))
spec_filter = np.loadtxt(os.path.join(homedir,"filter","filter_atlas_IR.txt"))
background = np.loadtxt(os.path.join(homedir,"background","backgrounds_mjy_per_str_ra_53.1_dec_-27.8.txt"))
userspec = np.loadtxt(os.path.join(os.getcwd(),"templates","Galaxy_Kmag_19_Haflux_5e-17_ebmv_0_z_2_highResSpec_obs_redshift.txt"))


mid_bkg = np.array([background[:,0] , background[:,2]])

#telescope_file = open(os.path.join(homedir,"telescope","telescope_data_atlas_IR.txt"),'r')
#telescope = telescope_file.read()
#maybe should put in dictionary first

telescope = {
            ## linearity limits of detector
            "one_per_limit": 55900,	# one percent non-linearity limit [e-]
            "five_per_limit":79550,	# five percent non-linearity limit [e-]
            "sat_limit":92450,	# saturation limit [e-]
            "AT":17671,	# Area of the telescope ((150/2.)**2 * np.pi) [cm2]
            "slit_W":0.75,	# slit width to measure R_theta [arcsec] > size of one micromirror
            "pix_scale":0.39,	# spatial pixel scale [arcsec/px]
            "tot_Npix":4096,	# Number of pixels in dispersion direction on the detector
            "det_RN":5,		# Detector readnoise (correlated double sampling) [e-/px]
            "Nmirror":2,		# Number of mirrors 
             ## Other stats
            "disp":10.0,	# dispersion [angstrom/px]
            "lambda":2.5,		# central wavelength [microns]
            "dark":0.005,	# dark current [e-/s]
            "Mref":1.0		# Mirror reflectance (reflectance already included in spectroscopic throughput)
            }
# adding one more variable into "telescope"
# R = lam / dlam
telescope["intrinsic_res"] = 1000 * telescope["slit_W"] 


In [34]:
#double check
#purpose is to mimic what output would look like based on the inherent flaws of optics of telescope
#basic process: convert everything to velocity, make a gaussian kernal and convolve it with the rest of the function, 
#               then convert back to wavelength

def degrade_resolution(wavelengths, flux, center_wave, spec_res, disp, px_tot):
    
    #allow more px for K band
    Npix_spec=px_tot * 3./2.

    #the speed of light in cm/s
    c=np.log10(29979245800.)
       # numerical convenience

    # make velocity array from -300,000 to 300,000
    vel=(np.arange(600001)-300000) #ok
    
    # convert wavelength to relative velocity
    in_vel=(wavelengths/center_wave-1)*10.**(1*c-5) 
    
    # create vectors in velocity space, picking out realistic values (keeping good indices)
    in_vel_short = in_vel[ np.where( (in_vel > vel[0]) & (in_vel < vel[600000]) )[0] ]
    in_flux_short = flux[ np.where( (in_vel > vel[0]) & (in_vel < vel[600000]) )[0] ] 
        
    # interpolate to 600000 points
    interp_flux = np.interp(vel, in_vel_short, in_flux_short)

    #sigma  = the resolution of the spectrograph
    sigma = (10.**(c-5)/spec_res)/(2*np.sqrt(2*np.log(2)))
        # convert to velocity resolution, potentially res per px


    # make a smaller velocity array with
    # the same "resolution" as the steps in
    # vel, above
    n = round(8.*sigma)
        # (potentially) making a kernel that is 8px 
    if (n % 2 == 0):
        n = n + 1
    vel_kernel = np.arange(n) - np.floor(n/2.0)

    # a gaussian of unit area and width sigma
    gauss_kernel = (1/(np.sqrt(2*np.pi)*sigma)) * np.exp(-0.5*vel_kernel**2.0/sigma**2.0)
        # shape the kernel
        # look up the equation for gaussian kernal and figure out the significance of sigma used here
        # like how does resolution shape/define the kernel

    # convolve flux with gaussian kernel
    convol_flux = np.convolve(interp_flux, gauss_kernel , mode="same") 
        # convolve moving kernel
    convol_wave = center_wave * (vel*10.**(-1*c+5.0) + 1.0 )
        # convert back to wavelength

    # and the real pixel scale of mosfire
    real_wave = np.arange(Npix_spec) * disp * 10.**(-4.)
    real_wave = real_wave - real_wave[int(np.round(Npix_spec/2.))]   
    real_wave = real_wave + center_wave 
        # wavelength to px

    # interpolate onto the pixel scale of the detector
    out_wave = real_wave
    out_flux = np.interp(real_wave , convol_wave, convol_flux)
        # interpolating to number of px (b/c working from km/px or lam/px)
    
    out = {"lam": out_wave,
          "flux": out_flux}
    
    return(out)
    

In [35]:
#functions 

def atlasIR_etc_serena(userinput):
    
    # General Constants 
    speedoflight = np.log10(29979245800) # speed of light in cm/2
    hplank = np.log10(6.626068)-27 # Plank constant in erg*s
    loghc = speedoflight + hplank # H*c in log(erg * cm)
    f_nu_AB = 48.59

    if userinput["nExp"] > 1:
        dither = 2.0
    else:
        dither = 1.0

    #Sky background conversion
    bkg = mid_bkg[1]
    bkg = bkg / 206265**2 # from MJy/sr to MJy/arcsec2
    bkg = bkg * 1e6 # from MJy/arcsec2 to Jy/arcsec2
    bkg = bkg * 1e-26 # from Jy/arcsec2 to W/m2/s/Hz/arcsec2
    bkg = bkg * 1e7 # from W/m2/s/Hz/arcsec2 to erg/m2/s/Hz/arcsec2
    bkg = bkg / (10**loghc / (background[:,0]*1e-4)) # from erg/m2/s/Hz/arcsec2 to ph/m2/s/Hz/arcsec2
    bkg = bkg * 10**speedoflight / (background[:,0]*1e-4)**2 # from ph/m2/s/Hz/arcsec2 to ph/m2/s/cm/arcsec2
    bkg = bkg * 1e-7 # from ph/m2/s/cm/arcsec2 to phm2/s/nm/arcsec2
    mid_bkg[1] = bkg

    
    # do the check single line input 
    # include conversion to micron from ang
    # include z shift
    # include check that input wavelength is within range
   
    #check line > if the user wants to know a specific line width (for what kind of object?)

    if (userinput["lineF"] > 0) & (userinput["lineW"] > 0) & (userinput["specFile"] == "none"):
        SPECTYPE = "line"

        # Convert from ang to micron
        if userinput["InputInAngstroms"] == True:
            userinput["lineW"] = userinput["lineW"]/10000


        # Put in observed frame
        center_w = userinput["lineW"] * (1+userinput["z"])
            # micron


        # Check to see if the band is correct
        if (center_w > np.nanmax(spec_filter[:,0])) | (center_w < np.nanmin(spec_filter[:,0])):
            print("This wavelength is not contained in the band")


    # check spectrum > if the user wants to view an entire spectrum: "A spectrum of a source with known emission line wavelengths is used to find the wavelength
    # scale (dispersion solution) for your spectra"
        # want to know: what wavelength is where
            # can use any kind of calibration source that you know the spec of, so you know where lam are supposed to appear (pix)
        # can also use to figure out best way to view a spectrum of an unknown source
    
    elif (userinput["specFile"] != "none") & (userinput["specFile"] != "flat"): 
        SPECTYPE = "user"
        if userinput["NormalizeUserSpec"] == True:
            if userinput["mag"] > 0:
                print("using 'mag' to normalize spectrum")
            else:
                print("The magnitude you gave for normalization doesn't make sense")

        else:
            if userinput["mag"] > 0:
                print("Your magnitude to normalize the spectrum looks reasonable but is not used because 'NormalizeUserSpec' is set to False.")            

        try: 
            user_spectrum = {"lam": userspec[:,0], "flux": userspec[:,1]}

        except:
            print("something went wrong loading the spectrum file")

        #convert to micron if needed
        if userinput["InputInAngstroms"] == True:
            user_spectrum["lam"] = user_spectrum["lam"] / 10000.0
                #this is now in MICRONS

        #put in observed frame
        user_spectrum["lam"] = user_spectrum["lam"] * (1+userinput["z"])
        user_spectrum["flux"] = user_spectrum["flux"] / (1+userinput["z"])
            # flux is in ergs/s/cm^2/Hz

        #make sure everything is in the right format (should match the filter)
        indicies = np.where(spec_filter[:,1] > 0.01)[0]
            # why 0.01? is it just an approximate number
        filter_check = spec_filter[:,1][indicies]

        #check if the lowest or highest are covered by the filter
        if (np.nanmin(user_spectrum["lam"]) > np.nanmin(filter_check) ) | ( np.nanmax(user_spectrum["lam"]) < np.nanmax(filter_check)  ):
            print("The spectrum you gave does not span the full wavelength coverage of the %s band or is not in the proper format. The correct format is observed-frame wavelength in microns or Angstroms and flux in erg/s/cm2, two column format. Please also check the wavelength unit.")
            quit()



    #check flat > for calibration purposes (difference: flat spectrum && flat illumination >> to figure out flaws of telescope to recalibrate)
        # this in particular is flat spectrum callibration (object with same AB mag across wavelengths)
    
    elif (userinput["specFile"] == "flat"):
        SPECTYPE = "flat"
        print("Using a flat spectrum in f_v (using mag=%g AB to normalize spectrum)!")
            # this is the standard object in AB

        if userinput["mag"] < 0:
            print("The magnitude you gave for normalization doesn't make sense" )
            quit()


    else:
        print("you're doing something wrong, don't know what you're looking for")


    ####NOTE: all of the arrays are np arrays so you can just multiply things
    
    
    
    ##-----throughput calculation-------##
    
    #throughput of telescope with mirrors taken into account
    throughput[:,1] = throughput[:,1] * telescope["Mref"] ** telescope["Nmirror"]

    # "real FWHM" > R = lam / dlam
    resolving_power = telescope["intrinsic_res"] / userinput["slit_width"]

    ##------degrade resolution-------##
    
    ##filter
    filter_degrade = degrade_resolution(wavelengths=spec_filter[:,0], 
                                        flux=spec_filter[:,1], 
                                        center_wave=telescope["lambda"], 
                                        spec_res=resolving_power, disp = telescope["disp"], 
                                        px_tot = telescope["tot_Npix"])

    #get the relevant portion (limits make sense after run through the degrade function)
    relevant_indices = np.where(filter_degrade['flux'] > 0.1)[0]
    filter_degrade["lam"] = np.array(filter_degrade["lam"][relevant_indices])
    filter_degrade["flux"] = np.array(filter_degrade["flux"][relevant_indices])

    filt_index = np.where(filter_degrade["flux"] > 0.5)[0]

    ##throughput
    throughput_degrade = degrade_resolution(wavelengths=throughput[:,0], 
                                        flux=throughput[:,1], 
                                        center_wave=telescope["lambda"], 
                                        spec_res=resolving_power, disp = telescope["disp"], 
                                        px_tot = telescope["tot_Npix"])
    #get relevant portions
    throughput_degrade["lam"] = throughput_degrade["lam"][relevant_indices]
    throughput_degrade["flux"] = throughput_degrade["flux"][relevant_indices]


    ##background
    background_degrade = degrade_resolution(wavelengths=mid_bkg[0], 
                                        flux=mid_bkg[1], 
                                        center_wave=telescope["lambda"], 
                                        spec_res=resolving_power, disp = telescope["disp"], 
                                        px_tot = telescope["tot_Npix"])
    #set negative values to 0
    where_zero = np.where(background_degrade["flux"] < 0)[0]
    background_degrade["flux"][where_zero] = 0

    background_degrade["lam"] = background_degrade["lam"][relevant_indices]
    background_degrade["flux"] = background_degrade["flux"][relevant_indices]


    
    ##-------Calculate spectrum----------
    
    
    if SPECTYPE == "line":

        #resolution at line:
        res_at_w = center_w / resolving_power

        #width of line before spectrograph
        width_b4_spec = center_w * userinput["FWHM"] * 10**(-speedoflight + 5)
            # convert to velocity


        #width of line after spectrograph
        width_final = np.sqrt(width_b4_spec**2 + res_at_w**2)
            #averaging the two noises

        #figure out the "location inside FWHM of given line"
        line_index = np.where(np.abs(background_degrade["lam"] - center_w) <= (0.5*width_final))[0]


        #check if chosen line is wide enough
        if len(line_index) == 0:
            print("ERROR: Line is too narrow.")
            quit()
        else:
            pass

        #area used to calculate SNR ??
        sn_index = line_index

        #recalculate background
        background_final = background_degrade["flux"] * throughput_degrade["flux"] * userinput["slit_width"] * userinput["theta"] * (telescope["AT"] * 10**(-4.)) * (telescope["disp"]/10.0)

        #interpolate background
        avg_bkg = np.interp(center_w,mid_bkg[0],mid_bkg[1])
                        # interp(where to interpolate, x-coordinates, y-coordinates)

        #convert into AB magnitude 
        bkg_mag = -2.5 * ( np.log10(avg_bkg*center_w) - 4 + 3 + hplank ) - f_nu_AB

        #signal after going through the atmosphere ?
        signal_atmos = userinput["lineF"] * 10**(-18-loghc-4) * center_w * telescope["AT"]
            # unit: photon/sec collected by telescope

        #width of line as sigma (micron)
        sigma = width_final / (2 * np.sqrt(2 * np.log(2)))

        #signal spectrum
        signal_spectrum = signal_atmos * ( 1 / ( np.sqrt(2 * np.pi) * sigma)) * np.exp(-0.5 * ( background_degrade["lam"] - center_w)**2 / sigma**2) * telescope["disp"] / 1e4
            # gaussian is 10 ang spacing, but per micron
            # / 1e4 to get photons/sec/ang
            # then multiply by ang/px to get photons/sec/px


        #signal speectrum in space
        signal_space = signal_spectrum * throughput_degrade["flux"]*1

        #number of px in the spectral direction
        npix_spec = (width_final * 10000.0) / telescope["disp"]
            # (width_final * 10000.0) == angstrom
            # npix_spec is in px


        #spatial pixel scale 
        npix_spatial_scale = userinput["theta"] / telescope["pix_scale"]


        # The number of pixels per FWHM observed
        npix_per_fwhm = npix_spatial_scale * npix_spec


    else:
        # The observed line wavelength in micron (here the center of the band from telescope)
            center_w = telescope["lambda"]
                # micron

            # resolution at the central wavelength in micron
            res_at_w = center_w / resolving_power
                # micron


            # the area used to calculate the S/N
            sn_index = filt_index

            # background magnitude
            bkg_mag = -2.5 * ( np.log10( np.nanmean(background_degrade["flux"][filt_index]) * center_w) - 4 + 3 + hplank) - f_nu_AB

            # now send the background spectrum through the telescope by 
            # multiplying the throughput, the
            # slit_width, the angular extent, the area of the
            # telescope, and the pixel scale in nm
            # this gives phot/sec/pixel
            background_final = background_degrade["flux"] * throughput_degrade["flux"] * userinput["slit_width"] * userinput["theta"] * (telescope["AT"] * 10.**(-4)) * (telescope["disp"] / 10.0)
                # phot/sec/px

            #-----For spectrum------:
            if SPECTYPE == "user": # for USER spectrum

                # convolve the user spectrum with the resolution
                user_spectrum_degrade = degrade_resolution(wavelengths=user_spectrum["lam"],
                                        flux=user_spectrum["flux"],
                                        center_wave=telescope["lambda"],
                                        spec_res=resolving_power,
                                        disp=telescope["disp"],
                                        px_tot=telescope["tot_Npix"]
                                        )

                user_spectrum_degrade["lam"] = user_spectrum_degrade["lam"][relevant_indices]
                user_spectrum_degrade["flux"] = user_spectrum_degrade["flux"][relevant_indices]
                    # userSig = flux, user_wave_grid = lam


                # multiply by a max-normalized filter transmission
                filt_shape = filter_degrade["flux"] / np.nanmax(filter_degrade["flux"])
                user_spectrum_degrade["flux"] = user_spectrum_degrade["flux"] * filt_shape

                # Check if User Spectrum needs to be normalized
                if userinput["NormalizeUserSpec"] == True:
                    # make the total match the broad band magnitude
                    scale = 10.0**(-0.4 * (userinput["mag"] + f_nu_AB)) / np.nanmean(user_spectrum_degrade["flux"])
                        #this is fv (look at wiki eq. for AB mag)
                        # this is to make the spectrum fainter (make the mags relative to AB standard)
                    raw_fv_sig_spec = user_spectrum_degrade["flux"] * scale

                else:
                    raw_fv_sig_spec = user_spectrum_degrade["flux"].copy()


                # convert to flux hitting the primary
                # in flux hitting the primary in phot/sec/micron 
                # (if the earth had no atmosphere)
                # phot/sec/micron = fnu * AT / lam / h
                signal_spectrum = raw_fv_sig_spec * 10.**(-1 * hplank) * telescope["AT"] / background_degrade["lam"]
                    #phot/sc/micron


            #------For flat------:

            elif SPECTYPE == "flat": ## for FLAT spectrum

                # flux hitting the primary in
                # phot/sec/micron (if the earth had no atmosphere)
                signal_spectrum = 10.0**(-0.4 * (userinput["mag"] + f_nu_AB) - hplank) * telescope["AT"] / background_degrade["lam"]

            # multiply by the atmospheric transparency (in space!)
            signal_spectrum = signal_spectrum * 1 # tranSpecObs

            # now put it through the throughput of the telescope
            # phot/sec/micron > phot/sec/pix
            signal_space = signal_spectrum * throughput_degrade["flux"] 

            # now for phot/sec/pix multiply by micron/pix
            signal_space = signal_space * (telescope["disp"] / 10000.0)            
                #phot/sec/pix

            # number of pixels per resolution element in the spectral direction
            npix_spec = (res_at_w*10000.0) / telescope["disp"]
                # = npix_spec

            # the spatial pixel scale
            # we have at least one pixel in the cross dispersion direction
            npix_spatial_scale = np.nanmax( np.asarray([userinput["theta"] / telescope["pix_scale"],2]) )
                #npix_spatial_scale

            # The number of pixels per FWHM observed
            npix_per_fwhm = npix_spec * npix_spatial_scale
                # = npix_per_fwhm




        
        
    
        ##-------------Calculate SNR, exposure, mag---------##
        
    #if not given exposure time:
    if userinput["SN"] > 0:

        #check if user line spectrum
        if (userinput["lineF"] > 0) & (userinput["lineW"] > 0):
            qa = -npix_spec * signal_space**2 / userinput["SN"]**2
        else:
            qa= -signal_space**2 / userinput["SN"]**2

        qb = dither * background_final + dither * telescope["dark"] * npix_spatial_scale + signal_space
        qc = dither * telescope["det_RN"]**2 / userinput["Nreads"] * npix_spatial_scale * userinput["nExp"]

  
        if len(sn_index) == 0:
            print("ERROR: No signal detected when computing exposure time for given input S/N.")
            quit()



        ##----calculate exposure time, assuming you know SNR----##
        exp_time_spec = (-qb[sn_index] - np.sqrt( qb[sn_index]**2 - 4 * qa[sn_index] * qc )) / (2 * qa[sn_index])
        exp_time = np.float( np.nanmedian( exp_time_spec) )



    else: ## Else, take the exposure time given by the user -----
        exp_time = userinput["time"]
    
    
    
    ##---compute SNR----##
    

    noise = np.sqrt( signal_space * exp_time + dither*((background_final + telescope["dark"] * npix_spatial_scale) * exp_time + telescope["det_RN"]**2 / userinput["Nreads"] * npix_spatial_scale * userinput["nExp"]) )
    signal_final =  signal_space * exp_time
        

    #SNR!
    SNR = signal_final / noise

    stn = np.nanmean(np.sqrt(npix_spec) * SNR[sn_index])
            # i think this means how many pixels is the noise?
            # this will print phot/s/spectral pix

    # the electron per pixel spectrum
    epp = noise**2 / npix_spatial_scale
        

    
    
    ##----follow the results printing pattern----##
    
    # the mean instrument+telescope throughput in the same band pass
    avg_throughput = np.nanmean( throughput_degrade["flux"][sn_index])

    # maximum electron per pixel
    max_epp = np.max( epp[sn_index] / userinput["nExp"] )


    # if calculating line flux ("S/N per FWHM ie S/N in the line")
    if (userinput["lineF"] > 0) & (userinput["lineW"] > 0): 

        # over the line (per FWHM)
        stn = np.nanmean(np.sqrt(npix_spec) * SNR[sn_index])
            #?

        # signal in e/FWHM
        signal_print = np.nanmean(signal_space[sn_index]) * npix_spec * exp_time
            #?

        # sky background in e/sec/FWHM
        background_print = np.nanmean( background_final[sn_index]) * npix_spec * exp_time

        # Read noise for multiple reads, electrons per FWHM
        RN_print = telescope["det_RN"] / np.sqrt(userinput["Nreads"]) * np.sqrt(npix_per_fwhm) * np.sqrt(userinput["nExp"])

        # noise per FWHM
        noise_print = np.nanmean( noise[sn_index]) * np.sqrt(npix_spec)

        # e- 
        dark_print = telescope["dark"] * npix_per_fwhm * exp_time

    # SNR per pixel for a continuous source
    else: 

        # per spectral pixel
        stn = np.nanmedian( SNR[sn_index] )

        # signal in e/(spectral pixel)
        signal_print = np.nanmedian( signal_space[sn_index]) * exp_time

        # sky background in e/(spectral pixel)
        background_print = np.nanmedian( background_final[sn_index]) * exp_time

        # Read noise for multiple reads, electrons per spectral pixel
        RN_print = telescope["det_RN"] / np.sqrt(userinput["Nreads"]) * np.sqrt(npix_spatial_scale) * np.sqrt(userinput["nExp"])

        # noise per spectral pixel
        noise_print = np.nanmedian(noise[sn_index])

        # e- per spectral pixel
        dark_print = telescope["dark"] * npix_spatial_scale * exp_time




        
    ##---------output----------##
    
    #Summary of results, in dictionary 
    summary_struct = dict()
    summary_struct["quant"] = ['Wavelength', 'Resolution','Dispersion', 'Throughput', 'Signal', 'Sky Background', 
                       'Sky brightness', 'Dark Current', 'Read Noise', 'Total Noise','S/N', 
                       'Total Exposure Time', 'Max e- per pixel']

    if (userinput["lineF"] > 0) & (userinput["lineW"] > 0):
        summary_struct["unit"] = ['micron','FWHM in angstrom', 'angstrom/pixel', '',  'electrons per FWHM',
         'electrons per FWHM', 'AB mag per sq. arcsec', 'electrons per FWHM', 
         'electrons per FWHM', 
         'electrons per FWHM',
         'per observed FWHM', 'seconds', 'electrons per pixel per exp']
    else:
        summary_struct["unit"] = ['micron','angstrom', 'angstrom/pixel', '',  'electrons per spectral pixel',
         'electrons per spectral pixel', 'AB mag per sq. arcsec', 'electrons per spectral pixel', 
         'electrons per spectral pixel', 'electrons per spectral pixel',
         'per spectral pixel', 'seconds', 'electrons per pixel']


    if max_epp >= 1e10:
        max_epp_string = "> 1e10"
    else:
        max_epp_string = max_epp


        
    #checking if the signal is saturating the detector
    if max_epp > telescope["sat_limit"]:
        print("Detector Saturated!")
    elif (max_epp > telescope["five_per_limit"]) & (max_epp < telescope["sat_limit"]):
        print("Detector in >5 percent unlinear regime")
    elif (max_epp > telescope["one_per_limit"]) & (max_epp < telescope["five_per_limit"]):
        print("Detector in 1 - 5 percent nonlinear regime")
    else:
        pass

    
    summary_struct["value"] = [round(center_w,4),
                    round(res_at_w * 1e4,1),
                    round(telescope["disp"],2),
                    avg_throughput,
                    signal_print,
                    background_print,
                    round(bkg_mag,6),
                    round(dark_print,6),
                    round(RN_print,6),
                    round(noise_print,6),
                    round(stn,6),
                    round(exp_time,6),
                    round(max_epp_string,6)
                    ]

    ## Actual output containing the spectrum (for graphing purposes) --------------
    spec_struct = dict()

    spec_struct["wave"] = background_degrade["flux"]
    spec_struct["center"] = center_w
    spec_struct["plot_index"] = sn_index
    spec_struct["filt_index"] = filt_index
    spec_struct["tp"] = throughput_degrade["flux"]
    spec_struct["filt"] = filter_degrade["flux"]
    spec_struct["bk"] = background_degrade["flux"]
    spec_struct["sig"] = signal_spectrum # phot/sec/micron
    spec_struct["signal"] = signal_final # phot/sec/micron
    spec_struct["noise"] = noise
    spec_struct["sn"] = SNR
    spec_struct["lineF"] = userinput["lineF"]
    spec_struct["time"] = exp_time

    output = {"summary_struct":summary_struct,
                "spec_struct":spec_struct}


    return(output)



    

In [36]:
# test out the function: test inputs come from example.py and README.md

userinput = {"band":"atlas_IR",
            "time":5000,
            "slit_width":0.75,
            "Nreads":16,
            "theta":0.7,
            "nExp":5,
            "lineF":-99,
            "lineW":-99,
            "FWHM":200,
            "z":0,
            "specFile":"./templates/Galaxy_Kmag_19_Haflux_5e-17_ebmv_0_z_2_highResSpec_obs_redshift.txt", 
            "mag":20,
            "NormalizeUserSpec":True,
            "InputInAngstroms":True,
            "SN":-99,
            }

etc_output = atlasIR_etc_serena(userinput)

fig = go.Figure(data=[go.Table(header=dict(values=['Variable', 'Value', 'Units']),
                 cells=dict(values=[
                     etc_output["summary_struct"]['quant'],
                     etc_output["summary_struct"]['value'],
                     etc_output["summary_struct"]['unit']]))
                     ])

fig.show()



using 'mag' to normalize spectrum


In [37]:
np.loadtxt("./templates/Galaxy_Kmag_19_Haflux_5e-17_ebmv_0_z_2_highResSpec_obs_redshift.txt")

array([[3.01500000e+03, 2.97578408e-28],
       [3.01650000e+03, 2.98129214e-28],
       [3.01800000e+03, 2.98680673e-28],
       ...,
       [2.99670000e+04, 1.16319974e-27],
       [2.99685000e+04, 1.16326851e-27],
       [2.99700000e+04, 1.16333726e-27]])